In [16]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import requests
import wikipedia
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re


# 1.0 Data Acquisition

Data will be acquired using two different approaches. First, movie datasets will be harvested from websites like Kaggle, Dataworld and statscrunch. Second, we will use the wikipedia API to extract basic movie information from the wikipedia page to fill in missing information about budget, running time and box office revenues. 

## 1.1. Reading directly from files
We will create pandas tables from structured datasets containing information about Academy Award nominations and wins, IMDB ratings and budget/box office 

In [ ]:
## Reading from cvs and excel files
df_actor = pd.read_csv('Movies/Oscar_data/actors.csv')
df_actress = pd.read_csv('Movies/Oscar_data/actresses.csv')
df_oscar = pd.read_csv('Movies/oscar_database.csv')
df_budget = pd.read_excel('Movies/statcrunch_budgetboxoffice.xlsx')
df_imdb = pd.read_excel('Movies/statcrunch_IMDB.xlsx')

# 1.2 Webscraping using wikipedia API

We will use the wikipedia API to obtain information about the cast running time, budget and box office revenues

In [17]:
# Define function fr obtaining movie data using omdb API
def omdbapi(title):
    if not isinstance(title, str):
        return {}
    
    url_base = 'http://www.omdbapi.com/?i=tt3896198&apikey=5db77b44&'
    url = url_base + 't=' + str(title)
    r = requests.get(url)
    json_data = r.json()
    return json_data

# Define function fr obtaining movie data using Wikipedia API
def wikiapi(title):
    title = title + '(film)'
    return wikipedia.page(title)

def wikiscrape(title):
    url = 'https://en.wikipedia.org/wiki/' + title
    my_client = urlopen(url)
    page_html = my_client.read()
    my_client.close()
    return page_html
    
    

In [27]:
movie = 'stardust_memories'
# x = omdbapi(movie)


In [36]:
# Using the wikipedia API to get the movie
# Just like the search field, we do not have to provide exact name
# We then parse the page using BeautifulSoup 
page = wikiapi(movie)
soup = BeautifulSoup(page.html(),'html.parser')


# Now find the infobox table within the page and extrcat it. 
# Ref: https://stackoverflow.com/questions/52913838/how-to-automate-scraping-wikipedia-info-box-specifically-and-print-the-data-usin?noredirect=1&lq=1
table = soup.find('table', attrs={'class': 'infobox vevent'})
infobox = table.find_all('tr')
infobox = str(infobox).lower()
print(type(infobox))
print(infobox)
print(len(infobox))




<class 'str'>
[<tr><th class="summary" colspan="2" style="text-align:center;font-size:125%;font-weight:bold;font-size:110%;font-style:italic;">stardust memories</th></tr>, <tr><td colspan="2" style="text-align:center"><a class="image" href="/wiki/file:stardust_memories_moviep.jpg"><img alt="stardust memories moviep.jpg" class="thumbborder" data-file-height="394" data-file-width="253" decoding="async" height="343" src="//upload.wikimedia.org/wikipedia/en/thumb/3/37/stardust_memories_moviep.jpg/220px-stardust_memories_moviep.jpg" srcset="//upload.wikimedia.org/wikipedia/en/3/37/stardust_memories_moviep.jpg 1.5x" width="220"/></a><div style="font-size:95%;padding:0.35em 0.35em 0.25em;line-height:1.25em;">theatrical release poster</div></td></tr>, <tr><th scope="row" style="white-space:nowrap;padding-right:0.65em;">directed by</th><td><a href="/wiki/woody_allen" title="woody allen">woody allen</a></td></tr>, <tr><th scope="row" style="white-space:nowrap;padding-right:0.65em;">produced by</

In [37]:
print(infobox[-20:])

4 million</td></tr>]


In [38]:
## Extracting box office revenues using regular expressions
pattern = re.compile(r'million')
matches = pattern.finditer(infobox)
for match in matches:
    print(match)



<re.Match object; span=(3283, 3290), match='million'>
<re.Match object; span=(3396, 3403), match='million'>
